In [1]:
import os
from glob import glob
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lsst.sims.maf.metricBundles as mb
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers

In [4]:
import sys
sys.path.append(os.path.join(os.getenv('SIMS_MAF_DIR'), 'bin.src'))
import movingObjects as moO

In [5]:
runname = 'astro-lsst-01_2013'
ddir = os.path.join('pha_split_res', runname)
outdir = runname + 'pha'
orbitFile = 'orbits/pha_5k'

In [6]:
slicer = slicers.MoObjSlicer()
slicer.readOrbits(orbitFile, Hrange=np.arange(11, 28.5, 0.5))

In [7]:
pha_splits = os.listdir('orbits/pha_split')
pha_splits

['pha_5k_00',
 'pha_5k_01',
 'pha_5k_02',
 'pha_5k_03',
 'pha_5k_04',
 'pha_5k_05',
 'pha_5k_06',
 'pha_5k_07',
 'pha_5k_08',
 'pha_5k_09']

In [8]:
# Read split results.
# ex: astro_lsst_01_1016_Discovery_Time_pha_5k_06_3_pairs_in_12_nights_MOOB.h5
strategy = ['3 pairs in 12 nights', '3 pairs in 15 nights', '3 pairs in 20 nights', 
            '3 pairs in 25 nights', '3 pairs in 30 nights', '4 pairs in 20 nights']
bundles = {}
for strat in strategy:
    bundles[strat] = {}
    for spl in pha_splits:
        filematch = os.path.join(ddir, runname + "_Discovery_Time*" + "*" + spl + "*" + 
                                 strat.replace(' ', '_') + "*h5")
        filename = glob(filematch)[0]
        bundles[strat][spl] = mb.createEmptyMoMetricBundle()
        bundles[strat][spl].read(filename)
        Hrange = bundles[strat][spl].slicer.Hrange
        bundles[strat][spl].slicer = slicer
        bundles[strat][spl].slicer.Hrange = Hrange

In [9]:
# Join results.
times = np.arange(59579, 65055.15+5, 10)
summaryMetrics = [metrics.MoCompletenessAtTimeMetric(times=times, Hval=22,cumulative=False, Hindex=0.33),
                  metrics.MoCompletenessAtTimeMetric(times=times, Hval=22, cumulative=True, Hindex=0.33)]


timebundles = {}
for strat in strategy:
    filematch = os.path.join(ddir, runname + "_Discovery_Time*" + "*pha_5k_00*" + strat.replace(' ', '_') + "*h5")
    filename = glob(filematch)[0]
    timebundles[strat] = mb.createEmptyMoMetricBundle()
    timebundles[strat].read(filename)
    timebundles[strat].fileRoot = timebundles[strat].fileRoot.replace('_00', '')
    Hrange1 = timebundles[strat].slicer.Hrange
    timebundles[strat].slicer = slicer
    timebundles[strat].slicer.Hrange = Hrange1
    mvals = np.zeros((len(slicer.orbits), len(Hrange1)), float)
    mvalsMask = np.ones((len(slicer.orbits), len(Hrange1)), dtype=bool)
    for spl in pha_splits:
        if np.any(np.not_equal(bundles[strat][spl].slicer.Hrange, Hrange1)):
            raise ValueError('Incompatible results, Hrange varies')
        mvals = mvals + bundles[strat][spl].metricValues.filled(0)
        mvalsMask = np.where(mvalsMask & bundles[strat][spl].metricValues.mask, True, False)
    timebundles[strat].metricValues = ma.MaskedArray(data=mvals, mask=mvalsMask, fill_value=0)
    timebundles[strat].metadata = 'PHA ' + strat
    timebundles[strat].runname = runname
    timebundles[strat].setSummaryMetrics(summaryMetrics)
    timebundles[strat].computeSummaryStats()
    timebundles[strat].write()

In [10]:
# old runs : 59580 start 
# v4 runs: 59853 start
59853 + 365 *10

63503

In [11]:
diff = np.abs(times-63503.50)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]

(array([392]), array([ 63499.]))

In [20]:
# astro-lsst-01-1016
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.74128294]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.75043703]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.7584721]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.76503545]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.77098024]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.7019879]


In [22]:
# astro-lsst-01_1016 @ time 10 years
diff = np.abs(times-63230.5)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63229.00'] [ 0.70072872]
3 pairs in 15 nights ['H <=22.0 @ 63229.00'] [ 0.71008376]
3 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.71860778]
3 pairs in 25 nights ['H <=22.0 @ 63229.00'] [ 0.72563463]
3 pairs in 30 nights ['H <=22.0 @ 63229.00'] [ 0.7321288]
4 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.65956568]


In [23]:
# astro-lsst-01_1016 @ time 15 years
diff = np.abs(times-65055.5)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 65059.00'] [ 0.78152201]
3 pairs in 15 nights ['H <=22.0 @ 65059.00'] [ 0.78909266]
3 pairs in 20 nights ['H <=22.0 @ 65059.00'] [ 0.79669691]
3 pairs in 25 nights ['H <=22.0 @ 65059.00'] [ 0.80288782]
3 pairs in 30 nights ['H <=22.0 @ 65059.00'] [ 0.80827317]
4 pairs in 20 nights ['H <=22.0 @ 65059.00'] [ 0.74443137]


In [177]:
# astro-lsst-01-1015
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.74067693]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.75258709]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.76580289]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.77648192]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.783748]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.68678541]


In [161]:
# astro-lsst-01-1017
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.75494339]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.76501288]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.77603137]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.7836424]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.78993808]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.71164474]


In [12]:
# minion_1016
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.69440213]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.70065299]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.70842489]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.71515535]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.72162124]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.65726505]


In [12]:
# astro-lsst-01_2013
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 63499.00'] [ 0.66976778]
3 pairs in 15 nights ['H <=22.0 @ 63499.00'] [ 0.6773945]
3 pairs in 20 nights ['H <=22.0 @ 63499.00'] [ 0.68375953]
3 pairs in 25 nights ['H <=22.0 @ 63499.00'] [ 0.68904486]
3 pairs in 30 nights ['H <=22.0 @ 63499.00'] [ 0.69425086]
4 pairs in 20 nights ['H <=22.0 @ 63499.00'] [ 0.635764]
